In [195]:
# prompt: connect to google drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [196]:
import ee
import os
import math

In [197]:
ee.Authenticate()
service_account = "lums-forest-fire-observatory@forest-fire-obs.iam.gserviceaccount.com"
# credentials = ee.ServiceAccountCredentials(service_account, "forest-fire-obs-5966ba29fefa.json")
credentials = ee.ServiceAccountCredentials(service_account, "/content/drive/MyDrive/google_earth_engine/forest-fire-obs-5966ba29fefa.json")
ee.Initialize(credentials=credentials, project='forest-fire-obs')
print(ee.String('Hello, Earth Engine!').getInfo())

Hello, Earth Engine!


In [198]:
def region_parameter(point, parameter,size):
  # Initialize an empty matrix with zeros
  matrix = [[0 for _ in range(size)] for _ in range(size)]

  x = ((math.floor(size/2))- size)*1000+1
  y = (size -(math.floor(size/2)))*1000+1

  # Fill the matrix with numbers according to the specified conditions
  for i in range(size):
    y = y -1000
    x = ((math.floor(size/2))- size)*1000+1
    for j in range(size):
      x = x+ 1000
      location = point.buffer(y,x).centroid()

      # Load the image collection
      collection = ee.ImageCollection("NOAA/GFS0P25")
      # Filter the collection by date
      filtered_collection = collection.filterDate('2024-01-01', '2024-01-02')
      # Select the parameter band
      parameter_band = parameter
      # Get the first image in the filtered collection
      image = filtered_collection.first()
      # Get the parameter value at the current point
      parameter_value = image.reduceRegion(ee.Reducer.first(), location).get(parameter_band).getInfo()

      # Add the parameter value to the matrix
      matrix[i][j] = parameter_value

  return matrix


In [199]:
# Define the original point of interest (Lahore, Pakistan)
original_point = ee.Geometry.Point(74.3587, 31.5204)
matrix_size = 32
# Get information for the square region around the original point for the parameter 'temperature_2m_above_ground'
region_info = region_parameter(original_point, 'temperature_2m_above_ground',matrix_size)

# Print the parameter values as a matrix
print("Parameter values for the square region around the original point:")
for row in region_info:
    print(row)

Parameter values for the square region around the original point:
[7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023]
[7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7.824853515625023, 7

In [200]:
# Load the image collection
collection = ee.ImageCollection("NOAA/GFS0P25")

# Get information about the first image in the collection
info = collection.first().getInfo()

# Print the properties of the image
print(info.keys())


dict_keys(['type', 'bands', 'version', 'id', 'properties'])


In [201]:
# Load the image collection
collection = ee.ImageCollection("NOAA/GFS0P25")

# Get information about the first image in the collection
info = collection.first().getInfo()

# Extract the bands information
bands_info = info['bands']

# Print the band IDs
for band in bands_info:
    print(band['id'])


temperature_2m_above_ground
specific_humidity_2m_above_ground
relative_humidity_2m_above_ground
u_component_of_wind_10m_above_ground
v_component_of_wind_10m_above_ground
total_precipitation_surface
precipitable_water_entire_atmosphere
total_cloud_cover_entire_atmosphere
downward_shortwave_radiation_flux


In [202]:
# Print the bands information
print("Bands:")
print(info['bands'])

# Print the properties information
print("Properties:")
print(info['properties'])


Bands:
[{'id': 'temperature_2m_above_ground', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [1441, 721], 'crs': 'GEOGCS["unknown", \n  DATUM["unnamed", \n    SPHEROID["Sphere", 6371229.0, 0.0]], \n  PRIMEM["Greenwich", 0.0], \n  UNIT["degree", 0.017453292519943295], \n  AXIS["Longitude", EAST], \n  AXIS["Latitude", NORTH]]', 'crs_transform': [0.25, 0, -180.125, 0, -0.25, 90.125]}, {'id': 'specific_humidity_2m_above_ground', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [1441, 721], 'crs': 'GEOGCS["unknown", \n  DATUM["unnamed", \n    SPHEROID["Sphere", 6371229.0, 0.0]], \n  PRIMEM["Greenwich", 0.0], \n  UNIT["degree", 0.017453292519943295], \n  AXIS["Longitude", EAST], \n  AXIS["Latitude", NORTH]]', 'crs_transform': [0.25, 0, -180.125, 0, -0.25, 90.125]}, {'id': 'relative_humidity_2m_above_ground', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [1441, 721], 'crs': 'GEOGCS["unknown", \n  DATUM["unnamed", \n 

In [203]:
# # Define the original point of interest (Lahore, Pakistan)
# original_point = ee.Geometry.Point(74.3587, 31.5204)

# # Define a point 1 km eastward from the original point
# eastward_point = original_point.buffer(1000).centroid()

# # Load the image collection
# collection = ee.ImageCollection("NOAA/GFS0P25")

# # Filter the collection by date
# filtered_collection = collection.filterDate('2024-01-01', '2024-01-02')

# # Select the temperature band
# temperature_band = 'temperature_2m_above_ground'

# # Get the first image in the filtered collection
# image = filtered_collection.first()

# # Get the temperature value at the original point
# temperature_original = image.reduceRegion(ee.Reducer.first(), original_point).get(temperature_band).getInfo()

# # Get the temperature value at the eastward point
# temperature_eastward = image.reduceRegion(ee.Reducer.first(), eastward_point).get(temperature_band).getInfo()

# # Print the temperature values
# print("Temperature at original point (Lahore), Pakistan (2024-01-01): {:.2f} °C".format(temperature_original - 273.15))
# print("Temperature at eastward point (1 km right of Lahore), Pakistan (2024-01-01): {:.2f} °C".format(temperature_eastward - 273.15))


In [204]:
# # Define a function to get information for a square region around a point
# def get_region_info(point, parameter):
#     # Compute the coordinates of surrounding points
#     north_point = point.buffer(1000).centroid()
#     south_point = north_point.buffer(-1000).centroid()
#     south_point = south_point.buffer(-1000).centroid()
#     east_point = point.buffer(0, 1000).centroid()
#     west_point = point.buffer(0, -1000).centroid()
#     northeast_point = north_point.buffer(0, 1000).centroid()
#     northwest_point = north_point.buffer(0, -1000).centroid()
#     southeast_point = south_point.buffer(0, 1000).centroid()
#     southwest_point = south_point.buffer(0, -1000).centroid()

#     # Load the image collection
#     collection = ee.ImageCollection("NOAA/GFS0P25")

#     # Filter the collection by date
#     filtered_collection = collection.filterDate('2024-01-01', '2024-01-02')

#     # Select the parameter band
#     parameter_band = parameter

#     # Get the first image in the filtered collection
#     image = filtered_collection.first()

#     # Get the parameter values for all points in the region
#     parameter_values = [
#         [image.reduceRegion(ee.Reducer.first(), northwest_point).get(parameter_band).getInfo() - 273.15,
#          image.reduceRegion(ee.Reducer.first(), north_point).get(parameter_band).getInfo() - 273.15,
#          image.reduceRegion(ee.Reducer.first(), northeast_point).get(parameter_band).getInfo() - 273.15],
#         [image.reduceRegion(ee.Reducer.first(), west_point).get(parameter_band).getInfo() - 273.15,
#          image.reduceRegion(ee.Reducer.first(), point).get(parameter_band).getInfo() - 273.15,
#          image.reduceRegion(ee.Reducer.first(), east_point).get(parameter_band).getInfo() - 273.15],
#         [image.reduceRegion(ee.Reducer.first(), southwest_point).get(parameter_band).getInfo() - 273.15,
#          image.reduceRegion(ee.Reducer.first(), south_point).get(parameter_band).getInfo() - 273.15,
#          image.reduceRegion(ee.Reducer.first(), southeast_point).get(parameter_band).getInfo() - 273.15]
#     ]

#     return parameter_values

# # Define the original point of interest (Lahore, Pakistan)
# original_point = ee.Geometry.Point(74.3587, 31.5204)

# # Get information for the square region around the original point for the parameter 'temperature_2m_above_ground'
# region_info = get_region_info(original_point, 'temperature_2m_above_ground')

# # Print the parameter values as a matrix
# print("Parameter values for the square region around the original point:")
# for row in region_info:
#     print(row)
